In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
df = pd.read_csv('ptrees_final_dataset.csv')
df.head()


,id,isin,ticker,name,marketname,currency,date,price,volume,book_value,...,sp_ratio,gross_profitability,capex_to_assets,debt_to_equity,sales_growth,cfo_to_assets,asset_quality,asset_growth,price_to_assets,asset_turnover
0,515411,CA00829Q1019,AOI.SE,Africa Oil Corp.,SSEFN,SEK,2010-09-30,10.2540,2.852158e+06,114333.352,...,1.069975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587634,0.628753
1,515411,CA00829Q1019,AOI.SE,Africa Oil Corp.,SSEFN,SEK,2010-10-29,12.2312,1.599629e+07,169053.716,...,0.897009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.700944,0.628753
2,515411,CA00829Q1019,AOI.SE,Africa Oil Corp.,SSEFN,SEK,2010-11-30,12.6910,9.819007e+06,169053.716,...,0.867652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.724660,0.628753
3,515411,CA00829Q1019,AOI.SE,Africa Oil Corp.,SSEFN,SEK,2010-12-30,12.4151,6.635532e+06,169053.716,...,0.887005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.708849,0.628753
4,515411,CA00829Q1019,AOI.SE,Africa Oil Corp.,SSEFN,SEK,2011-01-31,12.5071,1.270115e+07,208359.015,...,0.883720,-0.092327,0.360582,12789.22201,NaN,0.175829,-0.146841,NaN,0.711485,0.628753


### All the characteristics in the dataset

In [7]:
print(df.size)
df.columns

4215743


Index(['id', 'isin', 'ticker', 'name', 'marketname', 'currency', 'date',
       'price', 'volume', 'book_value', 'bookvaluecurrency', 'market_cap',
       'totalmarketvalue', 'marketvaluecurrency', 'year', 'month',
       'book_to_market', 'return_1m', 'momentum_12m', 'volatility_12m',
       'turnover', 'total_assets', 'net_income', 'total_revenue', 'cfo',
       'cogs', 'total_debt', 'capex', 'roa', 'ep_ratio', 'cfp_ratio',
       'sp_ratio', 'gross_profitability', 'capex_to_assets', 'debt_to_equity',
       'sales_growth', 'cfo_to_assets', 'asset_quality', 'asset_growth',
       'price_to_assets', 'asset_turnover'],
      dtype='object')

### Inflation rate from FRED. Based on CPI. Annual data

In [23]:
# Read CSV, treat first column as data instead of index
inflation = pd.read_csv("inflation rate sweden.csv", header=None, skiprows=2)

# Rename columns
inflation.columns = ["date", "inflation"]
# Convert 'date' column to datetime
inflation['date'] = pd.to_datetime(inflation['date'], dayfirst=True)  # adjust dayfirst if needed

# Filter for dates after 1996-12-31
inflation = inflation[inflation['date'] > '1996-12-31']
inflation.set_index('date', inplace=True)
inflation.tail()

,inflation
date,
2020-01-01,0.497367
2021-01-01,2.163197
2022-01-01,8.369291
2023-01-01,8.548625
2024-01-01,2.835817


### Volatility of Stock Price Index for Sweden from FRED. Volatility of stock price index is the 360-day standard deviation of the return on the national stock market index

In [21]:
volatility = pd.read_csv('Volatility of Stock Price Index for Sweden.csv', header=None, skiprows=2)
volatility.columns = ['date', 'volatility']
volatility['date'] = pd.to_datetime(volatility['date'], dayfirst=True)
volatility = volatility[volatility['date'] > '1996-12-31']
volatility.set_index('date', inplace=True)
volatility.tail()

,volatility
date,
2017-01-01,17.20446
2018-01-01,12.22559
2019-01-01,14.66502
2020-01-01,22.76061
2021-01-01,22.47797


### Option-adjusted spread is corporate yield - risk-free curve. Used as a proxy for default yield. OAS is in basis-points (1/100th of a percent)

In [18]:
OAS = pd.read_excel('OASpreadGraphExport.xls', header=None).iloc[54:2624]
OAS.columns = ['date', 'OAS']
OAS["date"] = pd.to_datetime(OAS["date"]).dt.date
OAS.set_index('date', inplace=True)

OAS

,OAS
date,
2015-11-02,120
2015-11-03,120
2015-11-04,118
2015-11-05,117
2015-11-06,108
...,...
2025-09-23,83
2025-09-24,86
2025-09-25,88


### Risk-free rate. Uses the Swedish 3-month treasury bill

In [25]:
rf = pd.read_csv(
    'Sweden 3 month Treasury rate monthly .csv',
    sep=',',
    header=1,       # skip first metadata row
    encoding='utf-8',
    engine='python'
)
# Keep only the relevant columns
rf = rf[['Datum', 'Senaste']].copy()
rf.rename(columns={'Datum': 'date', 'Senaste': 'Yield'}, inplace=True)

# Convert Date to datetime
rf['date'] = pd.to_datetime(rf['date'], dayfirst=True)

# Convert Swedish decimal commas to floats and turn percentages into decimals
rf['Yield'] = rf['Yield'].str.replace(',', '.').astype(float) / 100

rf = rf[rf['date'] > '1996-12-31']
# Set Date as index
rf.set_index('date', inplace=True)

# Check the cleaned data
print(rf.tail())


              Yield
date               
1997-01-05  0.04130
1997-01-04  0.04105
1997-01-03  0.04035
1997-01-02  0.04085
1997-01-01  0.03990


### Market return and volatility. Use return of OMX Stockholm 30

In [ ]:


# ------------------------------
# 1️⃣ Load first dataset (1998–2008)
# ------------------------------
market_return_1998 = pd.read_csv('OMX_30_from1998to2025.csv', header=1)
market_return_1998 = market_return_1998[['day', 'lastad']]
market_return_1998.rename(columns={'day': 'date', 'lastad': 'price'}, inplace=True)
market_return_1998['date'] = pd.to_datetime(market_return_1998['date'])
market_return_1998.sort_values('date', inplace=True)
market_return_1998.set_index('date', inplace=True)

# ------------------------------
# 2️⃣ Load second dataset (post 2008)
# ------------------------------
market_return_post_2008 = pd.read_excel('OMX_Stockholm_30_1998.xlsx')
market_return_post_2008 = market_return_post_2008[['Trade Date', 'Index Value']]
market_return_post_2008.rename(columns={'Trade Date': 'date', 'Index Value': 'price'}, inplace=True)
market_return_post_2008['date'] = pd.to_datetime(market_return_post_2008['date'])
market_return_post_2008.sort_values('date', inplace=True)
market_return_post_2008.set_index('date', inplace=True)

# ------------------------------
# 3️⃣ Combine datasets
# ------------------------------
market_return = pd.concat([market_return_1998, market_return_post_2008])
market_return.sort_index(inplace=True)

# ------------------------------
# 4️⃣ Clean data
# ------------------------------
market_return = market_return[market_return['price'] > 0]  # remove zero/negative prices
market_return = market_return[~market_return.index.duplicated()]  # remove duplicate dates

# ------------------------------
# 5️⃣ Ensure consecutive trading days
# ------------------------------
all_days = pd.date_range(start=market_return.index.min(), end=market_return.index.max(), freq='B')
market_return = market_return.reindex(all_days)
market_return['price'].ffill(inplace=True)  # forward-fill missing prices
market_return.index.name = 'date'

# ------------------------------
# 6️⃣ Compute daily returns
# ------------------------------
market_return['daily_return'] = market_return['price'].pct_change()
market_return = market_return[market_return['daily_return'].notna()]  # remove first NaN

# ------------------------------
# 7️⃣ Compute rolling 1-year (252 trading days) log returns
# ------------------------------
window = 252
market_return['rolling_log_return'] = np.log(market_return['price'] / market_return['price'].shift(window))
market_return['rolling_log_return_annualized'] = market_return['rolling_log_return'] * (252 / window)
market_return['rolling_log_return_annualized_pct'] = (market_return['rolling_log_return_annualized'] * 100).round(2)

# ------------------------------
# 8️⃣ Compute rolling volatility (annualized)
# ------------------------------
market_return['rolling_vol_daily'] = market_return['daily_return'].rolling(window).std()
market_return['rolling_vol_annualized'] = market_return['rolling_vol_daily'] * np.sqrt(252)
market_return['rolling_vol_annualized_pct'] = (market_return['rolling_vol_annualized'] * 100).round(2)

# ------------------------------
# 9️⃣ Filter only dates after 1996-12-31
# ------------------------------
market_return = market_return[market_return.index > '1996-12-31']

# ------------------------------
# 10️⃣ Inspect results
# ------------------------------
print(market_return[['price', 'rolling_log_return_annualized_pct', 'rolling_vol_annualized_pct']].head(10))
print(market_return[['price', 'rolling_log_return_annualized_pct', 'rolling_vol_annualized_pct']].tail(10))


              price  rolling_log_return_annualized_pct  \
date                                                     
1997-01-01  469.285                              35.50   
1997-01-02  462.852                              33.19   
1997-01-03  469.199                              35.14   
1997-01-06  469.199                              34.72   
1997-01-07  470.136                              34.76   
1997-01-08  480.142                              37.83   
1997-01-09  480.884                              37.48   
1997-01-10  473.841                              35.12   
1997-01-13  483.033                              36.48   
1997-01-14  488.207                              38.44   
1997-01-15  486.160                              37.69   
1997-01-16  487.552                              35.92   
1997-01-17  488.362                              35.09   
1997-01-20  487.299                              34.02   
1997-01-21  488.577                              33.24   
1997-01-22  49

/var/folders/1f/lczv_n1n1y19m0kx5q4l54840000gn/T/ipykernel_62806/2239305677.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  market_return['price'].ffill(inplace=True)  # forward-fill missing prices
